In [1]:
import os
import torch
from torch.utils.data import Dataset
import pandas as pd
from transformers import BertTokenizer, BertForMaskedLM
from nltk.tokenize import sent_tokenize
import nltk
import time
import pickle
from torch.utils.data import DataLoader

In [2]:
noteevents = pd.read_csv("/project/projectdirs/m1532/Projects_MVP/_datasets/mimiciii/NOTEEVENTS.csv")


/usr/common/software/python/3.7-anaconda-2019.10/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
final_df = pd.read_pickle("pickles/final_df.pickle")
notes_df = noteevents[["SUBJECT_ID","HADM_ID","CHARTDATE","CHARTTIME","CATEGORY","TEXT"]]
notes_df = final_df[["HADM_ID","SUBJECT_ID","DOB"]].drop_duplicates().merge(notes_df, on=["SUBJECT_ID","HADM_ID"], how="right")

notes = noteevents[["SUBJECT_ID","HADM_ID","CHARTDATE","CHARTTIME","CATEGORY","TEXT"]]
notes_df = pd.merge(final_df[["SUBJECT_ID","HADM_ID","DOB"]],notes.drop_duplicates(subset = ["SUBJECT_ID","HADM_ID","CHARTDATE","CHARTTIME"]), on=["SUBJECT_ID","HADM_ID"], how = "left")
notes_df = notes_df.drop_duplicates(subset='TEXT')
notes_df

,SUBJECT_ID,HADM_ID,DOB,CHARTDATE,CHARTTIME,CATEGORY,TEXT
0,109,173633,2117-08-07 00:00:00,2141-12-14,NaN,Discharge summary,Admission Date: [**2141-12-8**] ...
1,109,173633,2117-08-07 00:00:00,2141-12-07,NaN,ECG,Sinus tachycardia. No significant change comp...
2,109,173633,2117-08-07 00:00:00,2141-12-11,NaN,ECG,Sinus rhythm with first degree A-V block. Q-T...
3,109,173633,2117-08-07 00:00:00,2141-12-08,NaN,ECG,Sinus rhythm. No significant change compared ...
4,109,173633,2117-08-07 00:00:00,2141-12-09,2141-12-09 03:12:00,Nursing,Ms [**Known lastname 406**] is a 24 year old w...
...,...,...,...,...,...,...,...
370137,97492,189314,2126-09-04 00:00:00,2158-08-01,2158-08-01 10:17:00,Rehab Services,TITLE:\nBEDSIDE SWALLOWING EVALUATION:\nHISTOR...
370138,97492,189314,2126-09-04 00:00:00,2158-08-01,2158-08-01 00:34:00,Physician,SICU\n HPI:\n 31 yo RH woman with a PMH de...
370139,97492,189314,2126-09-04 00:00:00,2158-08-01,2158-08-01 10:43:00,Radiology,[**2158-8-1**] 10:43 AM\n MR HEAD W/O CONTRAST...
370140,97492,189314,2126-09-04 00:00:00,2158-07-31,2158-07-31 20:52:00,Radiology,[**2158-7-31**] 8:52 PM\n CTA HEAD W&W/O C & R...


In [4]:
salience_patient = pd.read_pickle("pickles/salience_patient.pickle")

In [5]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForMaskedLM.from_pretrained('bert-base-uncased')

In [ ]:
class textLoader(Dataset):
    def __init__(self,transform = None):
        self.input_ids_all = []
        self.attention_masks_all = []
        self.text = notes_df["TEXT"].tolist()
        self.samples = len(self.text)
        total = 0
        x=0
        for note in self.text:
            start = time.time()
            input_ids = []
            attention_masks = []
            try:
                for sent in sent_tokenize(note):
                    encoded_dict = tokenizer.encode_plus(
                        sent,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length = 64,           # Pad & truncate all sentences.
                        pad_to_max_length = True,
                        return_attention_mask = True,   # Construct attn. masks.
                        return_tensors = 'pt',     # Return pytorch tensors.
                   )
    
                    # Add the encoded sentence to the list.    
                    input_ids.append(encoded_dict['input_ids'])

                    # And its attention mask (simply differentiates padding from non-padding).
                    attention_masks.append(encoded_dict['attention_mask'])
            
                self.input_ids_all.append(torch.cat(input_ids, dim=0))
                self.attention_masks_all.append(torch.cat(attention_masks, dim=0))
            except:
                pass
            end = time.time()
            if((self.text.index(note)+1)%1000==0):
                x+=1
                total+=(end-start)
                print((self.text.index(note)+1)/367101, total/(x))
            
    def __getitem__(self,index):
        return self.input_ids_all[index]
    
    
    def __len__(self):
        return self.samples

    
data = textLoader()


In [7]:
with open('pickles/input_ids.pickle', 'wb') as f:
    pickle.dump(data, f)

/global/homes/a/ajaybati/.local/lib/python3.7/site-packages/torch/storage.py:34: FutureWarning: pickle support for Storage will be removed in 1.5. Use `torch.save` instead
  warnings.warn("pickle support for Storage will be removed in 1.5. Use `torch.save` instead", FutureWarning)


In [5]:
class textLoader(Dataset):
    def __init__(self,transform = None):
        self.input_ids_all = []
        self.attention_masks_all = []
        self.text = notes_df["TEXT"].tolist()
        self.samples = len(self.text)
        total = 0
        x=0
        for note in self.text:
            start = time.time()
            input_ids = []
            attention_masks = []
            try:
                for sent in sent_tokenize(note):
                    encoded_dict = tokenizer.encode_plus(
                        sent,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length = 64,           # Pad & truncate all sentences.
                        pad_to_max_length = True,
                        return_attention_mask = True,   # Construct attn. masks.
                        return_tensors = 'pt',     # Return pytorch tensors.
                   )
    
                    # Add the encoded sentence to the list.    
                    input_ids.append(encoded_dict['input_ids'])

                    # And its attention mask (simply differentiates padding from non-padding).
                    attention_masks.append(encoded_dict['attention_mask'])
            
                self.input_ids_all.append(torch.cat(input_ids, dim=0))
                self.attention_masks_all.append(torch.cat(attention_masks, dim=0))
            except:
                pass
            end = time.time()
            if((self.text.index(note)+1)%1000==0):
                x+=1
                total+=(end-start)
                print((self.text.index(note)+1)/367101, total/(x))
            
    def __getitem__(self,index):
        return self.input_ids_all[index]
    
    
    def __len__(self):
        return self.samples
    
    
with open('pickles/input_ids.pickle', 'rb') as f:
    data = pickle.load(f)

In [92]:
from torch.utils.data import TensorDataset, random_split

# Combine the training inputs into a TensorDataset.
dataset = TensorDataset(data.input_ids_all[0], data.attention_masks_all[0])

# Create a 90-10 train-validation split.

# Calculate the number of samples to include in each set.
train_size = int(0.9 * len(dataset))
val_size = len(dataset) - train_size

# Divide the dataset by randomly selecting samples.
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

print('{:>5,} training samples'.format(train_size))
print('{:>5,} validation samples'.format(val_size))

   16 training samples
    2 validation samples


In [24]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

# The DataLoader needs to know our batch size for training, so we specify it 
# here. For fine-tuning BERT on a specific task, the authors recommend a batch 
# size of 16 or 32.
batch_size = 32

# Create the DataLoaders for our training and validation sets.
# We'll take training samples in random order. 
train_dataloader = DataLoader(
            train_dataset,  # The training samples.
            sampler = RandomSampler(train_dataset), # Select batches randomly
            batch_size = batch_size # Trains with this batch size.
        )

# For validation the order doesn't matter, so we'll just read them sequentially.
validation_dataloader = DataLoader(
            val_dataset, # The validation samples.
            sampler = SequentialSampler(val_dataset), # Pull out batches sequentially.
            batch_size = batch_size # Evaluate with this batch size.
        )

In [88]:
masked_index = 2
sentence_inNOTE1 = 360000
word = 0
testing = data.input_ids_all[sentence_inNOTE1]
retokenize = [tokenizer.convert_ids_to_tokens(sent) for sent in testing]
real_sent = [tokenizer.convert_ids_to_tokens(sent) for sent in testing]
retokenize[word][masked_index] = '[MASK]'
retokenize = retokenize[word]
indexed_tokens = tokenizer.convert_tokens_to_ids(retokenize)
indexed_tokens = tokenizer.build_inputs_with_special_tokens(indexed_tokens)

tokens_tensor = torch.tensor([indexed_tokens])

loss, predictions = model(tokens_tensor, masked_lm_labels = tokens_tensor)

# confirm we were able to predict 'henson'
predicted_index = torch.argmax(predictions[0, masked_index+1]).item()
predicted_token = tokenizer.convert_ids_to_tokens([predicted_index])
print('predicted_token', predicted_token , loss)
print(retokenize2)
print("*"*80)
print(real_sent[word])

predicted_token ['day'] tensor(1.9854, grad_fn=<NllLossBackward>)
['[CLS]', 'demographics', '[MASK]', 'of', 'int', '##uba', '##tion', ':', 'day', 'of', 'mechanical', 'ventilation', ':', '14', 'ideal', 'body', 'weight', ':', '75', '.', '3', 'none', 'ideal', 'tidal', 'volume', ':', '301', '.', '2', '/', '451', '.', '8', '/', '60', '##2', '.', '4', 'ml', '/', 'kg', 'air', '##way', 'air', '##way', 'placement', 'data', 'known', 'difficult', 'int', '##uba', '##tion', ':', 'yes', 'tube', 'type', 'tr', '##ache', '##ost', '##omy', 'tube', ':', 'type', '[SEP]']
********************************************************************************
['[CLS]', 'demographics', 'day', 'of', 'int', '##uba', '##tion', ':', 'day', 'of', 'mechanical', 'ventilation', ':', '14', 'ideal', 'body', 'weight', ':', '75', '.', '3', 'none', 'ideal', 'tidal', 'volume', ':', '301', '.', '2', '/', '451', '.', '8', '/', '60', '##2', '.', '4', 'ml', '/', 'kg', 'air', '##way', 'air', '##way', 'placement', 'data', 'known', 'di